Imports

In [1]:
import PyPDF2
import logging
import re
import random
import torch
from tkinter import Tk, filedialog
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

c:\Users\d4848\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logging Setup

In [2]:
logging.basicConfig(
    filename="support_bot_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

Creating a Chat Bot

In [3]:
class SupportBotAgent:
    def __init__(self, use_embeddings=True, relevance_threshold=0.5):
        self.doc_text = ""
        self.sentences = []
        self.doc_embeddings = None
        self.use_embeddings = use_embeddings
        self.relevance_threshold = relevance_threshold
        self.model = SentenceTransformer("all-MiniLM-L6-v2") if use_embeddings else None
        self.qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
        self.gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
        self.gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
        self.generator = pipeline("text-generation", model=self.gpt2_model, tokenizer=self.gpt2_tokenizer)
        logging.info("SupportBotAgent initialized.")

    # Load Document
    def load_document(self, path):
        text = ""
        with open(path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    page_text = " ".join(page_text.split())  # clean line breaks
                    text += page_text + " "
        self.doc_text = text.strip()

        if self.use_embeddings and text.strip():
            sentences = [s for s in re.split(r'(?<=[.!?])\s+', text) if s.strip()]
            self.sentences = sentences
            self.doc_embeddings = self.model.encode(sentences, convert_to_tensor=True)

        logging.info("Document loaded: %s with %d characters", path, len(text))

    # Answer Query
    def answer_query(self, query):
        logging.info("Query received: %s", query)   # log query
        if not self.doc_text.strip():
            return "No document loaded."

        if not self.use_embeddings:
            return self.doc_text[:200] + "..."

    # Encode query & find best context
        query_embedding = self.model.encode(query, convert_to_tensor=True)
        cos_scores = util.cos_sim(query_embedding, self.doc_embeddings)[0]
        top_score, top_idx = torch.max(cos_scores, dim=0)

    # Relevance check
        if top_score < self.relevance_threshold:
            return "I don’t have enough information to answer that."

    # Use QA model
        context = self.sentences[top_idx]
        qa_result = self.qa_pipeline(question=query, context=context)
        answer = qa_result["answer"].strip()

    # If answer is too short, use GPT-2 to expand it
        if len(answer) < 20:
            prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
            gpt2_output = self.generator(
            prompt,
            max_length=150,
            num_return_sequences=1,
            do_sample=True
        )[0]["generated_text"]

        # Extract only the answer portion
            if "Answer:" in gpt2_output:
                gpt2_answer = gpt2_output.split("Answer:")[-1].strip()
            else:
                gpt2_answer = gpt2_output.strip()

            return gpt2_answer

    # If answer is already good length, just return it
        return answer


    # Simulate Feedback
    def simulate_feedback(self, response):
        feedback_options = ["not helpful", "too vague", "good"]
        feedback = random.choice(feedback_options)

        if feedback == "not helpful":
            response = response + " (rephrased for clarity)"
        elif feedback == "too vague":
            response = response + " (added more context)"
        # "good" -> keep as is

        logging.info("Feedback: %s | Adjusted Response: %s", feedback, response)
        return response, feedback

Main Fuction

In [4]:
if __name__ == "__main__":
    agent = SupportBotAgent()

    # File picker: choose ANY PDF from your PC
    Tk().withdraw()  # hide empty Tk window
    file_path = filedialog.askopenfilename(
        title="Select a PDF file",
        filetypes=[("PDF Files", "*.pdf")]
    )

    if not file_path:
        print("❌ No file selected.")
    else:
        agent.load_document(file_path)
        print("✅ Document loaded successfully!")

        while True:
            query = input("\nAsk a question (or type 'exit'): ")
            if query.lower() == "exit":
                print("\n🙏 Thanks for using the Support Bot!")
                print("✨ Hope I was able to help you today.")
                print("💬 Goodbye, and have a great day ahead! 🚀\n")
                break

    # Step 1: Answer query
            answer = agent.answer_query(query)
            print("💡 Answer:", answer)

    # Step 2: Refine with feedback (up to 10 iterations)
            for i in range(10):
                adjusted, fb = agent.simulate_feedback(answer)
                print(f"🔄 Iteration {i+1} | Feedback: {fb} | New Answer: {adjusted}")
                answer = adjusted
                if fb == "good":
                    break


Device set to use cpu
Device set to use cpu
Batches: 100%|██████████| 22/22 [00:04<00:00,  4.48it/s]


✅ Document loaded successfully!


Batches: 100%|██████████| 1/1 [00:00<00:00, 26.76it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


💡 Answer: Serri AI will replace the existing system of servers and make it easy to extend your games in any direction. Serri AI will make server managment much much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server managing much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server managment much more efficient. Serri AI will make server manag
🔄 Iteration 1 | Feedback: too vague | New Answer: Serri AI will replace the existing system of servers a

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.83it/s]


...

💡 Answer: Inform visitors about Serri's features and benefits
🔄 Iteration 1 | Feedback: too vague | New Answer: Inform visitors about Serri's features and benefits (added more context)
🔄 Iteration 2 | Feedback: not helpful | New Answer: Inform visitors about Serri's features and benefits (added more context) (rephrased for clarity)
🔄 Iteration 3 | Feedback: good | New Answer: Inform visitors about Serri's features and benefits (added more context) (rephrased for clarity)


Batches: 100%|██████████| 1/1 [00:00<00:00,  9.86it/s]


💡 Answer: Target Industries & Use Cases
🔄 Iteration 1 | Feedback: good | New Answer: Target Industries & Use Cases


Batches: 100%|██████████| 1/1 [00:00<00:00, 35.92it/s]


💡 Answer: Target Industries & Use Cases
🔄 Iteration 1 | Feedback: good | New Answer: Target Industries & Use Cases

🙏 Thanks for using the Support Bot!
✨ Hope I was able to help you today.
💬 Goodbye, and have a great day ahead! 🚀

